In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [2]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from skforecast.exceptions import ResidualsUsageWarning
from skforecast.direct import ForecasterDirect
from skforecast.direct import ForecasterDirectMultiVariate

# Fixtures
from skforecast.direct.tests.tests_forecaster_direct.fixtures_forecaster_direct import y
from skforecast.direct.tests.tests_forecaster_direct.fixtures_forecaster_direct import exog, exog_predict
# from skforecast.direct.tests.tests_forecaster_direct_multivariate.fixtures_forecaster_direct_multivariate import series
# from skforecast.direct.tests.tests_forecaster_direct_multivariate.fixtures_forecaster_direct_multivariate import exog, exog_predict


transformer_exog = ColumnTransformer(
                       [('scale', StandardScaler(), ['exog_1']),
                        ('onehot', OneHotEncoder(), ['exog_2'])],
                       remainder = 'passthrough',
                       verbose_feature_names_out = False
                   )

In [3]:
forecaster = ForecasterDirect(LinearRegression(), steps=5, lags=3)
forecaster.fit(y=y)
results = forecaster._predict_interval_conformal(
    steps=5, nominal_coverage=0.95, use_in_sample_residuals=True, use_binned_residuals=True
)
results.to_numpy()

array([[ 0.51883519,  0.08813429,  0.94953609],
       [ 0.4584716 ,  0.04326184,  0.87368136],
       [ 0.39962743, -0.04758052,  0.84683537],
       [ 0.40452904, -0.0426789 ,  0.85173698],
       [ 0.41534557, -0.03186238,  0.86255351]])

In [4]:
results

,pred,lower_bound,upper_bound
50,0.518835,0.088134,0.949536
51,0.458472,0.043262,0.873681
52,0.399627,-0.047581,0.846835
53,0.404529,-0.042679,0.851737
54,0.415346,-0.031862,0.862554


In [5]:
results[['pred', 'lower_bound', 'upper_bound']].to_numpy()

array([[ 0.51883519,  0.08813429,  0.94953609],
       [ 0.4584716 ,  0.04326184,  0.87368136],
       [ 0.39962743, -0.04758052,  0.84683537],
       [ 0.40452904, -0.0426789 ,  0.85173698],
       [ 0.41534557, -0.03186238,  0.86255351]])

In [2]:
import re
import pytest
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from scipy.stats import norm

from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.recursive import ForecasterRecursive
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import backtesting_forecaster_multiseries
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.preprocessing import RollingFeatures

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection_multiseries import series

In [7]:
forecaster = ForecasterDirectMultiVariate(
                     regressor          = Ridge(random_state=123),
                     level              = 'l1',
                     lags               = 3,
                     steps              = 5,
                     transformer_series = None,
                     binner_kwargs      = {'n_bins': 10}
                 )
    
cv = TimeSeriesFold(
            initial_train_size = len(series) - 20,
            steps              = 5,
            refit              = False
        )

metrics_levels, backtest_predictions = backtesting_forecaster_multiseries(
                                            forecaster              = forecaster,
                                            series                  = series,
                                            cv                      = cv,
                                            levels                  = 'l1',
                                            metric                  = 'mean_absolute_error',
                                            add_aggregated_metric   = False,
                                            exog                    = series['l1'].rename('exog_1'),
                                            interval                = 0.9,
                                            interval_method         = 'conformal',
                                            use_in_sample_residuals = True,
                                            use_binned_residuals    = True,
                                            random_state            = 123,
                                            verbose                 = False
                                        )

  0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
metrics_levels.to_numpy()

array([['l1', 0.11916192143057436]], dtype=object)

In [9]:
backtest_predictions[['pred', 'lower_bound', 'upper_bound']].to_numpy()

array([[0.36268205, 0.20433986, 0.52102424],
       [0.51787041, 0.43620381, 0.599537  ],
       [0.52472422, 0.42645324, 0.6229952 ],
       [0.59903598, 0.46654146, 0.7315305 ],
       [0.50738853, 0.42572194, 0.58905513],
       [0.42622724, 0.3313424 , 0.52111209],
       [0.47163992, 0.38997332, 0.55330651],
       [0.70873165, 0.48281375, 0.93464954],
       [0.71565768, 0.48973979, 0.94157557],
       [0.49000773, 0.40834113, 0.57167433],
       [0.61296286, 0.48046834, 0.74545738],
       [0.4031027 , 0.30821786, 0.49798755],
       [0.39728743, 0.30240258, 0.49217227],
       [0.46010284, 0.37843624, 0.54176944],
       [0.63022554, 0.49773102, 0.76272006],
       [0.39528807, 0.30040323, 0.49017292],
       [0.47347775, 0.39181115, 0.55514435],
       [0.76236728, 0.53644939, 0.98828517],
       [0.53407054, 0.43579957, 0.63234152],
       [0.5014202 , 0.4197536 , 0.5830868 ]])

In [4]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from skforecast.recursive import ForecasterRecursiveMultiSeries

# Fixtures
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import series, exog, exog_predict

THIS_DIR = r"C:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\tests\tests_forecaster_recursive_multiseries"
series_dict = joblib.load(THIS_DIR + '/fixture_sample_multi_series.joblib')
exog_dict = joblib.load(THIS_DIR + '/fixture_sample_multi_series_exog.joblib')
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}
series_2 = pd.DataFrame({
    'l1': pd.Series(np.arange(10)),
    'l2': pd.Series(np.arange(10))
})


In [51]:
forecaster = ForecasterRecursiveMultiSeries(
                     regressor          = LGBMRegressor(
                         n_estimators=30, random_state=123, verbose=-1, max_depth=4
                     ),
                     lags               = 14,
                     encoding           = 'ordinal',
                     dropna_from_series = False,
                     transformer_series = StandardScaler(),
                     transformer_exog   = StandardScaler()
                 )
forecaster.fit(
    series=series_dict_train, exog=exog_dict_train, suppress_warnings=True
)

levels = ['id_1000', 'id_1001', 'id_1003', 'id_1004', 'id_1005']
last_window = pd.DataFrame(
    {k: v for k, v in forecaster.last_window_.items() if k in levels}
)
last_window['id_1005'] = last_window['id_1004'] * 0.9
exog_dict_test_2 = exog_dict_test.copy()
exog_dict_test_2['id_1005'] = exog_dict_test_2['id_1001']
results = forecaster.predict_interval(
        steps=5, levels=levels, exog=exog_dict_test_2, last_window=last_window,
        suppress_warnings=True, n_boot=10, interval=[5, 95], use_binned_residuals=True
    )

In [52]:
results.head(2)

,level,pred,lower_bound,upper_bound
2016-08-01,id_1000,1330.538536,1280.691397,1439.455380
2016-08-01,id_1001,2655.952531,2705.925594,2738.156981


In [53]:
pd.Index(results.index)

DatetimeIndex(['2016-08-01', '2016-08-01', '2016-08-01', '2016-08-01',
               '2016-08-01', '2016-08-02', '2016-08-02', '2016-08-02',
               '2016-08-02', '2016-08-02', '2016-08-03', '2016-08-03',
               '2016-08-03', '2016-08-03', '2016-08-03', '2016-08-04',
               '2016-08-04', '2016-08-04', '2016-08-04', '2016-08-04',
               '2016-08-05', '2016-08-05', '2016-08-05', '2016-08-05',
               '2016-08-05'],
              dtype='datetime64[ns]', freq=None)

In [54]:
results.to_dict(orient='list')

{'level': ['id_1000',
  'id_1001',
  'id_1003',
  'id_1004',
  'id_1005',
  'id_1000',
  'id_1001',
  'id_1003',
  'id_1004',
  'id_1005',
  'id_1000',
  'id_1001',
  'id_1003',
  'id_1004',
  'id_1005',
  'id_1000',
  'id_1001',
  'id_1003',
  'id_1004',
  'id_1005',
  'id_1000',
  'id_1001',
  'id_1003',
  'id_1004',
  'id_1005'],
 'pred': [1330.5385359512072,
  2655.9525305801844,
  2645.090876890802,
  7897.5193849420975,
  4890.228408881278,
  1401.6308515684586,
  2503.7524796063676,
  2407.1752505407007,
  8577.098408560274,
  4756.810200061664,
  1387.265728819639,
  2446.2803866452205,
  2314.0860223813092,
  8619.983117285357,
  4947.440527174498,
  1310.8227594194243,
  2389.3764241040913,
  2245.0514974677576,
  8373.803343371283,
  4972.509186937418,
  1279.3727451240927,
  2185.0610428355444,
  2197.452881661525,
  8536.318209938987,
  5213.761246797846],
 'lower_bound': [1280.691397121706,
  2705.9255940079975,
  2351.2169926457227,
  7456.30218140561,
  4518.09207378455

In [55]:
forecaster = ForecasterRecursiveMultiSeries(
        regressor=LGBMRegressor(
            n_estimators=30, random_state=123, verbose=-1, max_depth=4
        ),
        lags=[1, 7, 14],
        encoding='ordinal',
        dropna_from_series=False,
        transformer_series=None,
        transformer_exog=StandardScaler(),
    )

cv = TimeSeriesFold(
            initial_train_size = len(series_dict_train['id_1000']),
            steps              = 24,
            refit              = False
        )

metrics, predictions = backtesting_forecaster_multiseries(
    forecaster           = forecaster,
    series               = series_dict,
    exog                 = exog_dict,
    cv                   = cv,
    metric               = ['mean_absolute_error', 'mean_absolute_scaled_error'],
    interval             = 0.9,
    interval_method      = "conformal",
    use_binned_residuals = True,
    n_jobs               = 'auto',
    verbose              = False,
    show_progress        = True,
    suppress_warnings    = True
)

  0%|          | 0/7 [00:00<?, ?it/s]

In [57]:
metrics.to_dict(orient='list')

{'levels': ['id_1000',
  'id_1001',
  'id_1002',
  'id_1003',
  'id_1004',
  'average',
  'weighted_average',
  'pooling'],
 'mean_absolute_error': [177.94640447766702,
  1451.3480109896332,
  nan,
  277.78113362955673,
  993.6769068120083,
  725.1881139772163,
  724.9604804988818,
  724.960480498882],
 'mean_absolute_scaled_error': [0.8178593233613526,
  4.1364664709651064,
  nan,
  1.1323827428361022,
  0.8271748048818786,
  1.72847083551111,
  2.0965105153721213,
  1.760615501057647]}

In [59]:
predictions.head(10).to_dict(orient='list')

{'level': ['id_1000',
  'id_1001',
  'id_1003',
  'id_1004',
  'id_1000',
  'id_1001',
  'id_1003',
  'id_1004',
  'id_1000',
  'id_1001'],
 'pred': [1559.6918278739745,
  2934.363291873329,
  3392.6095502790795,
  7097.054479229451,
  1572.8044765344362,
  3503.7475024125847,
  3118.0493908325134,
  8301.53364484904,
  1537.6749468304602,
  3354.2752034001587],
 'lower_bound': [1336.857463880777,
  2569.1917715600575,
  2556.6487812039504,
  5093.465893848657,
  1349.9701125412387,
  3138.575982099313,
  2282.0886217573843,
  7895.559245312443,
  1314.8405828372627,
  2989.103683086887],
 'upper_bound': [1782.526191867172,
  3299.534812186601,
  4228.570319354209,
  9100.643064610245,
  1795.6388405276336,
  3868.9190227258564,
  3954.0101599076424,
  8707.508044385637,
  1760.5093108236576,
  3719.4467237134304]}